In [120]:
## build researcher events, student events, and self-study events
## researcher are simply those with NH citations

In [121]:
import sys

import pandas as pd
import numpy as np
import time
import datetime
import os
from nanoHUB.application import Application
from nanoHUB.configuration import ClusteringConfiguration
from nanoHUB.pipeline.geddes.data import get_default_s3_client
from nanoHUB.dataaccess.lake import S3FileMapper

from nanoHUB.clustering.infra import get_mike_only_clustered_users, get_xufeng_only_clustered_users, get_mike_xufeng_clustered_users
from nanoHUB.clustering.infra import create_clusters_repository, add_cluster_info, get_all_clustered_users, get_all_self_study_users, get_all_self_identified_users

In [122]:
pd.set_option('display.max_columns', None)
cwd = os.getcwd()

In [123]:
## determine starting year
#if cache_flag == False:
start_year_base = 1999 #2000
start_year_base = 2012 #2000
pathsave=cwd+'/cached_data/'
pathsave=cwd+'/test_data/'


#else:
#     print('check the saved file')
#    start_year_base = int(user_breakdown_df['year_month'].to_list()[-1][:4])
    
    

In [124]:
application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')
wang159_myrmekes_db = application.new_db_engine('rfm_data')

s3_client = get_default_s3_client(application)
derived_data_file_path = 'derived_data_for_users.csv'
processed_mapper = S3FileMapper(s3_client, ClusteringConfiguration().bucket_name_processed)

In [125]:
def Annualized_Usage(nanohub_metrics_db,start_year,list_usernames,list_title,pathsave):
    today = datetime.datetime.today()
    print(today)    
    #cache_r_track = pd.DataFrame()
    #cache_si_track = pd.DataFrame()
    #cache_si_track['user'] = self_identified_track
    active_track = []
    active_date = []
    
    ## computer has memory limits, so split toolstart into branches
    # this filters and finds the research usage in nanohub
    while start_year < today.year+1:
        start_year += 1
        end_date = r"'"+str(start_year)+r"-01-01'"
        start_date = r"'"+str(start_year-2)+r"-01-01'"

        print('end date: '+ str(end_date))
        for i in range(1,13):
            if i < 10:
                start_date = r"'"+str(start_year-1)+r"-0"+str(i)+r"-01'"
                end_date = r"'"+str(start_year)+r"-0"+str(i)+r"-01'"
            else:
                start_date = r"'"+str(start_year-1)+r"-"+str(i)+r"-01'"
                end_date = r"'"+str(start_year)+r"-"+str(i)+r"-01'"

            sql_query = "select user from toolstart where user in " + str(tuple(list_usernames))\
                +" and datetime <= "+end_date+" and datetime >= "+start_date
            active_users = pd.read_sql_query(sql_query, nanohub_metrics_db)
            #print('date '+str(end_date)+' '+list_title+' partial',active_users.drop_duplicates()[0:10])
            print('date '+str(end_date)+' '+list_title+' length',len(active_users.drop_duplicates()))

            #count the number of active users and append into an array
            active_track.append(active_users.drop_duplicates().shape[0])
            active_date.append(end_date)


        print('end date: '+ str(end_date))
        #display(researcher_track)
        #display(researcher_track)

        cache_track = pd.DataFrame()
        cache_track['track'] = active_track
        cache_track.to_csv(pathsave+list_title+'_track.csv')
        cache_track.to_csv(pathsave+ str(start_year)+'_'+list_title+'_track.csv')
        cache_track = []
        
        cache_datetrack = pd.DataFrame()
        cache_datetrack['date'] = active_date
        cache_datetrack['track'] = active_track
        cache_datetrack.to_csv(pathsave+list_title+'_date_track.csv')
        cache_datetrack.to_csv(pathsave+ str(start_year)+'_'+list_title+'_date_track.csv')
        cache_datetrack = []
        

    return active_track

## Cumulative classroom/research/self-study

In [126]:
clustered_df = get_all_clustered_users(processed_mapper, derived_data_file_path)
clustered_df['nanoHUB_user_ID__c'] = clustered_df['id']
clustered_NH_ids = clustered_df['nanoHUB_user_ID__c']
display(len(clustered_NH_ids))

78167

In [127]:
mike_clustered_users_df = get_mike_only_clustered_users(processed_mapper, derived_data_file_path)
mike_clustered_users_df['nanoHUB_user_ID__c'] = mike_clustered_users_df['id']
mike_clustered_NH_ids = mike_clustered_users_df['nanoHUB_user_ID__c']
display(len(mike_clustered_users_df))

[WARNING] [warnings - py.warnings]: /home/gekco/nanoHUB/nanoHUB/clustering/infra.py:334: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return df[df[column_name].str.contains(search_for)]
 [warnings._showwarnmsg:109]


16825

In [128]:
xufeng_clustered_users_df = get_xufeng_only_clustered_users(processed_mapper, derived_data_file_path)
xufeng_clustered_users_df['nanoHUB_user_ID__c'] = xufeng_clustered_users_df['id']
xufeng_clustered_NH_ids = xufeng_clustered_users_df['nanoHUB_user_ID__c']
display(len(xufeng_clustered_users_df))

14997

In [129]:
mike_xufeng_clustered_users_df = get_mike_xufeng_clustered_users(processed_mapper, derived_data_file_path)
mike_xufeng_clustered_users_df['nanoHUB_user_ID__c'] = mike_xufeng_clustered_users_df['id']
mike_xufeng_clustered_NH_ids = mike_xufeng_clustered_users_df['nanoHUB_user_ID__c']
display(len(mike_xufeng_clustered_users_df))

46345

In [130]:
# raise

In [131]:
self_study_df = get_all_self_study_users(processed_mapper, derived_data_file_path)
self_study_df['nanoHUB_user_ID__c'] = self_study_df['id']
self_study_NH_ids = self_study_df['nanoHUB_user_ID__c']
self_study_usernames = self_study_df['username'].tolist()

print('self_study_NH_ids length = ',len(self_study_NH_ids))
print('self_study_usernames length = ',len(self_study_usernames))
print('self_study_usernames samples = ',self_study_usernames[0:10])

self_study_NH_ids length =  194524
self_study_usernames length =  194524
self_study_usernames samples =  ['hubrepo', 'admin', 'support', 'gridstat', 'ncn', 'nanohub', 'repo', 'violin', 'mireles', 'k_dadesh']


In [132]:
# self_identified users, are individuals that have a researcherID, google scholar etc in their profile
self_identified_df = get_all_self_identified_users(processed_mapper, derived_data_file_path)
self_identified_df['nanoHUB_user_ID__c'] = self_identified_df['id']
self_identified_NH_ids = self_identified_df['nanoHUB_user_ID__c']
self_identified_usernames = self_identified_df['username'].tolist()
print('length of self_identified_NH_ids',len(self_identified_NH_ids))
display(self_identified_df.head())



length of self_identified_NH_ids 4037


,id,name,username,email,registerDate,lastvisitDate,first_citation_at,number_citations,profile_key,has_researcher_profile,number_simulations,uid,clusters,mike_fall_2008,xufeng_fall_2008,mike_spring_2008,xufeng_spring_2008,mike_fall_2009,xufeng_fall_2009,mike_spring_2009,xufeng_spring_2009,mike_fall_2010,xufeng_fall_2010,mike_spring_2010,xufeng_spring_2010,mike_fall_2011,xufeng_fall_2011,mike_spring_2011,xufeng_spring_2011,mike_fall_2012,xufeng_fall_2012,mike_spring_2012,xufeng_spring_2012,mike_fall_2013,xufeng_fall_2013,mike_spring_2013,xufeng_spring_2013,mike_fall_2014,xufeng_fall_2014,mike_spring_2014,xufeng_spring_2014,mike_fall_2015,xufeng_fall_2015,mike_spring_2015,xufeng_spring_2015,mike_fall_2016,xufeng_fall_2016,mike_spring_2016,xufeng_spring_2016,mike_fall_2017,xufeng_fall_2017,mike_spring_2017,xufeng_spring_2017,mike_fall_2018,xufeng_fall_2018,mike_spring_2018,xufeng_spring_2018,mike_fall_2019,xufeng_fall_2019,mike_spring_2019,xufeng_spring_2019,mike_fall_2020,xufeng_fall_2020,mike_spring_2020,xufeng_spring_2020,mike_fall_2021,xufeng_fall_2021,nanoHUB_user_ID__c
895,2579,Stefan Birner,birner,stefan.birner@nextnano.com,2006-05-05 07:36:09,2020-01-13 15:58:33,NaN,0.0,[orcid],True,52.0,2579.0,['mike_spring_2010'],False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2579
1176,2862,Mark Lundstrom,lundstro,lundstro@purdue.edu,2004-12-06 22:01:25,2022-02-08 16:28:36,NaN,0.0,"[googlescholar, researchgateid]",True,2345.0,2862.0,"['xufeng_fall_2008', 'xufeng_spring_2008', 'xu...",False,True,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,2862
1473,3159,Tamara J. Moore,tmoore,tmoore@purdue.edu,2003-09-03 21:18:50,2021-10-07 19:58:59,NaN,0.0,[orcid],True,38.0,3159.0,['xufeng_fall_2020'],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,3159
1796,3482,Gerhard Klimeck,gekco,gekco@purdue.edu,2004-04-10 17:58:53,2022-02-10 14:45:37,NaN,0.0,"[orcid, googlescholar, researcherid, researchg...",True,18677.0,3482.0,"['xufeng_fall_2009', 'xufeng_spring_2009', 'xu...",False,False,False,False,False,True,False,True,False,True,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,3482
1897,3583,Eric Pop,epop,epop@stanford.edu,2006-02-09 20:57:14,2021-09-04 07:19:40,NaN,0.0,"[orcid, googlescholar]",True,90.0,3583.0,['xufeng_fall_2012'],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,3583


In [133]:
allusers_df = processed_mapper.read(derived_data_file_path)
display(allusers_df.head(5))
#display(allusers_usernames)
# we know there are some users who have a NULL user name - they need to be taken out
display(allusers_df.loc[allusers_df['username'].isnull()])
#display(allusers_df.loc[allusers_df['username'].notna()])

# take out the users with a NULL user name
allusers_df = allusers_df.loc[allusers_df['username'].notna()]
display(allusers_df.loc[allusers_df['username'].isnull()])

# create a list of all user names
allusers_usernames = allusers_df['username'].tolist()

[WARNING] [warnings - py.warnings]: /tmp/ipykernel_6286/952934691.py:1: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  allusers_df = processed_mapper.read(derived_data_file_path)
 [warnings._showwarnmsg:109]


,id,name,username,email,registerDate,lastvisitDate,first_citation_at,number_citations,profile_key,has_researcher_profile,number_simulations,uid,clusters,mike_fall_2008,xufeng_fall_2008,mike_spring_2008,xufeng_spring_2008,mike_fall_2009,xufeng_fall_2009,mike_spring_2009,xufeng_spring_2009,mike_fall_2010,xufeng_fall_2010,mike_spring_2010,xufeng_spring_2010,mike_fall_2011,xufeng_fall_2011,mike_spring_2011,xufeng_spring_2011,mike_fall_2012,xufeng_fall_2012,mike_spring_2012,xufeng_spring_2012,mike_fall_2013,xufeng_fall_2013,mike_spring_2013,xufeng_spring_2013,mike_fall_2014,xufeng_fall_2014,mike_spring_2014,xufeng_spring_2014,mike_fall_2015,xufeng_fall_2015,mike_spring_2015,xufeng_spring_2015,mike_fall_2016,xufeng_fall_2016,mike_spring_2016,xufeng_spring_2016,mike_fall_2017,xufeng_fall_2017,mike_spring_2017,xufeng_spring_2017,mike_fall_2018,xufeng_fall_2018,mike_spring_2018,xufeng_spring_2018,mike_fall_2019,xufeng_fall_2019,mike_spring_2019,xufeng_spring_2019,mike_fall_2020,xufeng_fall_2020,mike_spring_2020,xufeng_spring_2020,mike_fall_2021,xufeng_fall_2021
0,998,hubrepo hubrepo,hubrepo,nkissebe@gmail.com,2014-11-13 21:09:09,NaN,NaN,0.0,[],False,0.0,NaN,[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1000,CMS Admin Manager,admin,support@nanohub.org,NaN,NaN,NaN,0.0,[],False,0.0,NaN,[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,1683,nanoHUB support,support,support@nanohub.org,2008-11-19 22:51:04,2008-11-19 23:55:30,NaN,0.0,[],False,2.0,1683.0,[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,1684,Grid Statistics,gridstat,gridstat@nanohub.org,2008-11-18 17:29:56,2020-02-14 18:50:14,NaN,0.0,[],False,0.0,NaN,[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,1685,NCN NCN,ncn,ncn@nanohub.org,2008-11-11 19:17:04,NaN,NaN,0.0,[],False,0.0,NaN,[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


,id,name,username,email,registerDate,lastvisitDate,first_citation_at,number_citations,profile_key,has_researcher_profile,number_simulations,uid,clusters,mike_fall_2008,xufeng_fall_2008,mike_spring_2008,xufeng_spring_2008,mike_fall_2009,xufeng_fall_2009,mike_spring_2009,xufeng_spring_2009,mike_fall_2010,xufeng_fall_2010,mike_spring_2010,xufeng_spring_2010,mike_fall_2011,xufeng_fall_2011,mike_spring_2011,xufeng_spring_2011,mike_fall_2012,xufeng_fall_2012,mike_spring_2012,xufeng_spring_2012,mike_fall_2013,xufeng_fall_2013,mike_spring_2013,xufeng_spring_2013,mike_fall_2014,xufeng_fall_2014,mike_spring_2014,xufeng_spring_2014,mike_fall_2015,xufeng_fall_2015,mike_spring_2015,xufeng_spring_2015,mike_fall_2016,xufeng_fall_2016,mike_spring_2016,xufeng_spring_2016,mike_fall_2017,xufeng_fall_2017,mike_spring_2017,xufeng_spring_2017,mike_fall_2018,xufeng_fall_2018,mike_spring_2018,xufeng_spring_2018,mike_fall_2019,xufeng_fall_2019,mike_spring_2019,xufeng_spring_2019,mike_fall_2020,xufeng_fall_2020,mike_spring_2020,xufeng_spring_2020,mike_fall_2021,xufeng_fall_2021
77305,82487,Geraldine Young,NaN,f948941@rmqkr.net,2013-05-22 20:04:55,2013-06-14 20:51:07,NaN,0.0,[],False,39.0,82487.0,"['mike_fall_2008', 'xufeng_fall_2008', 'mike_s...",True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
211599,281303,Nataly Cárdenas,NaN,njcardenasvasquez@gmail.com,2020-03-20 17:26:07,2020-03-20 17:26:07,NaN,0.0,[],False,4.0,281303.0,"['mike_fall_2008', 'xufeng_fall_2008', 'mike_s...",True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


,id,name,username,email,registerDate,lastvisitDate,first_citation_at,number_citations,profile_key,has_researcher_profile,number_simulations,uid,clusters,mike_fall_2008,xufeng_fall_2008,mike_spring_2008,xufeng_spring_2008,mike_fall_2009,xufeng_fall_2009,mike_spring_2009,xufeng_spring_2009,mike_fall_2010,xufeng_fall_2010,mike_spring_2010,xufeng_spring_2010,mike_fall_2011,xufeng_fall_2011,mike_spring_2011,xufeng_spring_2011,mike_fall_2012,xufeng_fall_2012,mike_spring_2012,xufeng_spring_2012,mike_fall_2013,xufeng_fall_2013,mike_spring_2013,xufeng_spring_2013,mike_fall_2014,xufeng_fall_2014,mike_spring_2014,xufeng_spring_2014,mike_fall_2015,xufeng_fall_2015,mike_spring_2015,xufeng_spring_2015,mike_fall_2016,xufeng_fall_2016,mike_spring_2016,xufeng_spring_2016,mike_fall_2017,xufeng_fall_2017,mike_spring_2017,xufeng_spring_2017,mike_fall_2018,xufeng_fall_2018,mike_spring_2018,xufeng_spring_2018,mike_fall_2019,xufeng_fall_2019,mike_spring_2019,xufeng_spring_2019,mike_fall_2020,xufeng_fall_2020,mike_spring_2020,xufeng_spring_2020,mike_fall_2021,xufeng_fall_2021


In [134]:
# ideally we would not search activities for all users but only in the active users.
# so here we could determine the list of all active users to make the queries for total sim users over time 
# run faster
# 

#sql_query_simtotal = "select distinct user from toolstart;"
#simtotal_users = pd.read_sql_query(sql_query_simtotal, nanohub_metrics_db) #engine_metrics #.drop_duplicates()


In [135]:
#print('simtotal_users length', len(simtotal_users))
#display(simtotal_users)


In [136]:
print('clustered users',clustered_NH_ids.shape)
print('selfstudy users',self_study_NH_ids.shape)
print('self_study_usernames',len(self_study_usernames))
print('self identified researchers',self_identified_NH_ids.shape)
print('all users',allusers_df.shape)

clustered users (78167,)
selfstudy users (194524,)
self_study_usernames 194524
self identified researchers (4037,)
all users (272689, 67)


In [137]:
frames = [clustered_df, self_study_df]
contacts_df = pd.concat(frames)

In [138]:
## need to filter self-study-nh-ids with the researchers

In [139]:
## This researcher determination is based entirely on the results of research authors on nanohub
## i.e., we use the researcher information on db2

In [140]:
sql_query = 'select id, uid, author from jos_citations'
jos_citations = pd.read_sql_query(sql_query, nanohub_db)
display(jos_citations.head(2))
print(jos_citations.shape)

,id,uid,author
0,10000001,5568,"Weber, Bent; Mahapatra, Suddhasatta; Ryu, Hoon..."
1,10000002,5568,"Andrawis, Robert; Bermeo, Jose; Charles, James..."


(4203, 3)


In [141]:
t1_authors = jos_citations['author'].to_list()
t1_authors2 = [] #[j.split(';') for i,j in enumerate(t1_authors)]

In [142]:
for i,j in enumerate(t1_authors):
    try:
        holder = j.split(';')
        if len(holder) > 1: 
            for k,l in enumerate(holder):
                if l[0] == ' ':
                    holder[k] = l[1:]
        t1_authors2.append(holder)
    except:
        garb = True

In [143]:
t1_authors32 = [item for sublist in t1_authors2 for item in sublist]
t1_authors3 = set(t1_authors32)
print(len(t1_authors3))

6860


In [144]:
## extract all researchers with nh ids
rw_ids = []
need_search = []
for i,j in enumerate(t1_authors3):
    if '{{' in j:
        rw_ids.append(i)
    else:
        need_search.append(i)

In [145]:
## from the rw_ids, extract the nanohub userids
# can obtain usernames from the DB2 table
rw_ids2 = np.array(list(t1_authors3))[rw_ids]
for i,j in enumerate(rw_ids2):
    begin = j.index('{{')
    rw_ids2[i] = j[begin+2:-2]
print(len(rw_ids2))

2079


In [146]:
# need to obtain usernames
sql_query = "select id,name,username from jos_users where id in "+str(tuple(rw_ids2))
rw_researchers = pd.read_sql_query(sql_query,nanohub_db)
display(rw_researchers.head(2))

,id,name,username
0,1742,H.-S. Philip Wong,hspwong
1,1767,Osama Munir Nayfeh,onayfeh


In [147]:
## need to obtain the username/ids from those people without nh ids explicitly spec'd in jos_citations
ns2 = np.array(list(t1_authors3))[need_search]
ns2 = ns2[1:]

In [148]:
tuple(ns2)[:5]

('M. Zargham', 'Holger Buch', 'TH Darma', 'Abbas Kouzani', 'R.V. Joshi')

In [149]:
sql_query = "select id,name,username from jos_users where name in "+str(tuple(ns2))
ns2_researchers = pd.read_sql_query(sql_query,nanohub_db)
display(ns2_researchers.head(2))

,id,name,username
0,282167,Aarthi Narayanan,aarthin0102
1,280280,Abani Patra,apatra01


In [150]:
## combine the usernames into one data structure
researcher_usernames = rw_researchers['username'].to_list() + ns2_researchers['username'].to_list()
print('list of researchers who have cited nanohub  researcher_usernames=',len(researcher_usernames))

list of researchers who have cited nanohub  researcher_usernames= 3046


In [151]:
#need to take out the researchers and citing authors from the list of selfstudy users. 
#
print('self_identified_usernames can have overlap with citing or self identified researchers',len(self_identified_usernames))
print('take out overlap of len(researcher_usernames)',len(researcher_usernames))
print('take from len(self_identified_usernames)',len(self_identified_usernames))

print('number of self_identified_usernames users start',len(self_study_usernames))
self_identified_usernames = [x for x in self_identified_usernames if x not in researcher_usernames]
print('number of self_identified_usernames users end',len(self_study_usernames))

print('number of self-study users self_study_usernames with research',len(self_study_usernames))
self_study_usernames = [x for x in self_study_usernames if x not in researcher_usernames]
print('number of self-study users self_study_usernames without research',len(self_study_usernames))

print('number of self-study users self_study_usernames with self identified research',len(self_study_usernames))
self_study_usernames = [x for x in self_study_usernames if x not in self_identified_usernames]
print('number of self-study users self_study_usernames without self identified research',len(self_study_usernames))

self_identified_usernames can have overlap with citing or self identified researchers 4037
take out overlap of len(researcher_usernames) 3046
take from len(self_identified_usernames) 4037
number of self_identified_usernames users start 194524
number of self_identified_usernames users end 194524
number of self-study users self_study_usernames with research 194524
number of self-study users self_study_usernames without research 192480
number of self-study users self_study_usernames with self identified research 192480
number of self-study users self_study_usernames without self identified research 189161


In [152]:
today = datetime.datetime.today()
print(today)

2022-02-22 17:38:38.819258


In [153]:
#print(user_breakdown_df['year_month'].to_list()[-1][:4])

In [154]:
from copy import deepcopy

In [155]:
all_months_list = list()
cache_flag = False
for this_year in range(start_year_base+1,today.year+2):
    if cache_flag == True:
        for this_month in range(1,today.month+1):
            all_months_list.append(datetime.datetime(this_year, this_month, 1))
    else:
        for this_month in range(1,12+1):
            all_months_list.append(datetime.datetime(this_year, this_month, 1))

cache_all_month_list = pd.DataFrame()
cache_all_month_list['Date'] = all_months_list
cache_all_month_list.to_csv(pathsave+'all_months_list.csv')
cache_all_month_list.to_csv(pathsave+ str(start_year_base)+'_all_months_list.csv')
cache_all_month_list = []
print('allmonths partial list',all_months_list[0:36])


allmonths partial list [datetime.datetime(2013, 1, 1, 0, 0), datetime.datetime(2013, 2, 1, 0, 0), datetime.datetime(2013, 3, 1, 0, 0), datetime.datetime(2013, 4, 1, 0, 0), datetime.datetime(2013, 5, 1, 0, 0), datetime.datetime(2013, 6, 1, 0, 0), datetime.datetime(2013, 7, 1, 0, 0), datetime.datetime(2013, 8, 1, 0, 0), datetime.datetime(2013, 9, 1, 0, 0), datetime.datetime(2013, 10, 1, 0, 0), datetime.datetime(2013, 11, 1, 0, 0), datetime.datetime(2013, 12, 1, 0, 0), datetime.datetime(2014, 1, 1, 0, 0), datetime.datetime(2014, 2, 1, 0, 0), datetime.datetime(2014, 3, 1, 0, 0), datetime.datetime(2014, 4, 1, 0, 0), datetime.datetime(2014, 5, 1, 0, 0), datetime.datetime(2014, 6, 1, 0, 0), datetime.datetime(2014, 7, 1, 0, 0), datetime.datetime(2014, 8, 1, 0, 0), datetime.datetime(2014, 9, 1, 0, 0), datetime.datetime(2014, 10, 1, 0, 0), datetime.datetime(2014, 11, 1, 0, 0), datetime.datetime(2014, 12, 1, 0, 0), datetime.datetime(2015, 1, 1, 0, 0), datetime.datetime(2015, 2, 1, 0, 0), datetime

In [156]:
# clustered user usernames
tc_query = 'select id,username from jos_users where id in ' +str(tuple(clustered_NH_ids))
clustered_NH_username = pd.read_sql_query(tc_query,nanohub_db)
display(len(clustered_NH_username))

tc_query = 'select id,username from jos_users where id in ' +str(tuple(mike_clustered_NH_ids))
mike_clustered_NH_username = pd.read_sql_query(tc_query,nanohub_db)
display(len(mike_clustered_NH_username))

tc_query = 'select id,username from jos_users where id in ' +str(tuple(xufeng_clustered_NH_ids))
xufeng_clustered_NH_username = pd.read_sql_query(tc_query,nanohub_db)
display(len(xufeng_clustered_NH_username))

tc_query = 'select id,username from jos_users where id in ' +str(tuple(mike_xufeng_clustered_NH_ids))
mike_xufeng_clustered_NH_username = pd.read_sql_query(tc_query,nanohub_db)
display(len(mike_xufeng_clustered_NH_username))



78167

16825

14997

46345

In [157]:
## self study user usernames
#ts_query = 'select id,username from jos_users where id in ' + str(tuple(self_study_NH_ids.dropna().to_list())) 
##sf_contact_ids['nanoHUB_user_ID__c']))
#ss_NH_username = pd.read_sql_query(ts_query,nanohub_db)

In [158]:
## cache the researchers, self-study, and clustered users
cache_r_nhid = pd.DataFrame()
#cache_r_nhid['nhid'] = researcher_ids
cache_r_nhid['username'] = researcher_usernames

cache_si_nhid = pd.DataFrame()
#cache_si_nhid['nhid'] = self_identified_NH_ids.to_list()
cache_si_nhid['username'] = self_identified_usernames

cache_ss_nhid = pd.DataFrame()
#cache_ss_nhid['nhid'] = ss_NH_username['id']
cache_ss_nhid['username'] = self_study_usernames

cache_c_nhid = pd.DataFrame()
cache_c_nhid['nhid'] = clustered_NH_username['id']
cache_c_nhid['username'] = clustered_NH_username['username']

cache_cm_nhid = pd.DataFrame()
cache_cm_nhid['nhid'] = mike_clustered_NH_username['id']
cache_cm_nhid['username'] = mike_clustered_NH_username['username']

cache_cx_nhid = pd.DataFrame()
cache_cx_nhid['nhid'] = xufeng_clustered_NH_username['id']
cache_cx_nhid['username'] = xufeng_clustered_NH_username['username']

cache_cmx_nhid = pd.DataFrame()
cache_cmx_nhid['nhid'] = mike_xufeng_clustered_NH_username['id']
cache_cmx_nhid['username'] = mike_xufeng_clustered_NH_username['username']

In [159]:
print(cache_r_nhid.head(2))
print(cache_si_nhid.head(2))
print(cache_ss_nhid.head(2))
print(cache_c_nhid.head(2))
print(cache_cm_nhid.head(2))
print(cache_cx_nhid.head(2))
print(cache_cmx_nhid.head(2))

  username
0  hspwong
1  onayfeh
  username
0   tmoore
1  alexwei
  username
0  hubrepo
1    admin
     nhid    username
0  166284  0.yao.yuan
1  344801  007sribabu
   nhid username
0  1767  onayfeh
1  1909  reimers
   nhid username
0  1848   kevdog
1  1852  himadri
   nhid      username
0  1794        mar889
1  2136  ashtone_rose


In [160]:
cache_r_nhid.to_csv(pathsave+'citingauthor_ids.csv')
cache_si_nhid.to_csv(pathsave+'selfidentifiedauthors_ids.csv')
cache_ss_nhid.to_csv(pathsave+'unclassified_ids.csv')
cache_c_nhid.to_csv(pathsave+'clustered_all_ids.csv')
cache_cm_nhid.to_csv(pathsave+'clustered_m_ids.csv')
cache_cx_nhid.to_csv(pathsave+'clustered_x_ids.csv')
cache_cmx_nhid.to_csv(pathsave+'clustered_mx_ids.csv')

In [161]:
#call the annual number analysis as a function
start_year = start_year_base
#clustered_track       = Annualized_Usage(nanohub_metrics_db,start_year,clustered_NH_username['username'],'clustered_all',pathsave)
#clustered_m_track     = Annualized_Usage(nanohub_metrics_db,start_year,mike_clustered_NH_username['username'],'clustered_m',pathsave)
#clustered_x_track     = Annualized_Usage(nanohub_metrics_db,start_year,xufeng_clustered_NH_username['username'],'clustered_x',pathsave)
clustered_mx_track    = Annualized_Usage(nanohub_metrics_db,start_year,mike_xufeng_clustered_NH_username['username'],'clustered_mx',pathsave)
#researcher_track      = Annualized_Usage(nanohub_metrics_db,start_year,researcher_usernames,'citingauthor',pathsave)
#self_identified_track = Annualized_Usage(nanohub_metrics_db,start_year,self_identified_usernames,'selfidentifiedauthors',pathsave)
#self_study_track      = Annualized_Usage(nanohub_metrics_db,start_year,self_study_usernames,'unclassified',pathsave)
#simtotal_track        = Annualized_Usage(nanohub_metrics_db,start_year,allusers_usernames,'simtotal',pathsave)



2022-02-22 17:38:44.907937
end date: '2013-01-01'
date '2013-01-01' clustered_mx length 3477
date '2013-02-01' clustered_mx length 3530
date '2013-03-01' clustered_mx length 3617
date '2013-04-01' clustered_mx length 3654
date '2013-05-01' clustered_mx length 3507
date '2013-06-01' clustered_mx length 3409
date '2013-07-01' clustered_mx length 3385
date '2013-08-01' clustered_mx length 3317
date '2013-09-01' clustered_mx length 3470
date '2013-10-01' clustered_mx length 3930
date '2013-11-01' clustered_mx length 3945
date '2013-12-01' clustered_mx length 3982
end date: '2013-12-01'
end date: '2014-01-01'
date '2014-01-01' clustered_mx length 3704
date '2014-02-01' clustered_mx length 3814
date '2014-03-01' clustered_mx length 3896
date '2014-04-01' clustered_mx length 3803
date '2014-05-01' clustered_mx length 3608
date '2014-06-01' clustered_mx length 3591
date '2014-07-01' clustered_mx length 3610
date '2014-08-01' clustered_mx length 3557
date '2014-09-01' clustered_mx length 3542
d

In [162]:
print('clustered_track ',len(clustered_track))
print('clustered_m_track ', len(clustered_m_track))
print('clustered_x_track ',len(clustered_x_track))
print('clustered_mx_track ',len(clustered_mx_track))
print('researcher_track ',len(researcher_track))
print('self_identified_track ', len(self_identified_track))
print('self_study_track ',len(self_study_track))
print('simtotal_track ',len(simtotal_track))
print('all_months_list ',len(all_months_list))


NameError: name 'clustered_track' is not defined